## Reading files


We will read the `.txt` files line by line and apply these filters:

1. **Remove lines containing a WhatsApp encryption notice**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.`
    - ✅ **After:** (Removed)

2. **Remove lines with `<Media omitted>`**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: <Media omitted>`
    - ✅ **After:** (Removed)

3. **Remove lines containing email addresses**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: example@gmail.com`
    - ✅ **After:** (Removed)

4. **Remove lines containing links**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: https://www.example.com/`
    - ✅ **After:** (Removed)

5. **Replace `<This message was edited>` with an empty string**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: hey, how are you? <This message was edited>`
    - ✅ **After:** `dd/mm/yyyy, hh:mm - Person: hey, how are you?`

6. **Remove lines with the text `You deleted this message`**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: You deleted this message`
    - ✅ **After:** (Removed)

7. **Remove lines with the text `null`**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: null`
    - ✅ **After:** (Removed)

8. **Remove lines with the text `created group`**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person created group "group name"`
    - ✅ **After:** (Removed)

9. **Remove lines with the text `added you`**

    - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person added you`
    - ✅ **After:** (Removed)

10. **Replace tagging (`@person`) with an empty string**

-   ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: @person are you coming?`
-   ✅ **After:** `dd/mm/yyyy, hh:mm - Person: are you coming?`


In [18]:
import re
import pandas as pd


def read_whatsapp_chat(file_path: str) -> pd.DataFrame:
    # Define filtering patterns
    encryption_message = "Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more."
    media_pattern = "<Media omitted>"
    email_pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}'
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    edited_message = "<This message was edited>"
    deleted_message = "You deleted this message"
    null_message = "null"
    created_group_message = "created group"
    added_you_to_group_message = "added you"
    tagging_pattern = r'@[\w]+'

    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Apply filters to remove unwanted lines
    filtered_lines = []
    for line in lines:
        if (
            encryption_message not in line and
            deleted_message not in line and
            null_message != line.split(" ")[-1] and
            media_pattern not in line and
            created_group_message not in line and
            added_you_to_group_message not in line and
            not re.search(email_pattern, line) and
            not re.search(url_pattern, line)
        ):
            line = line.replace(edited_message, "").strip()
            line = re.sub(tagging_pattern, "", line).strip()
            filtered_lines.append(line)

    # Regular expression to match WhatsApp message format
    pattern = r'(\d{2}/\d{2}/\d{4}, \d{2}:\d{2}) - (.*?): (.*?)(?=\n\d{2}/\d{2}/\d{4}, \d{2}:\d{2} -|$)'
    content = '\n'.join(filtered_lines)
    messages = re.findall(pattern, content, re.DOTALL)

    df = pd.DataFrame(messages, columns=['timestamp', 'sender', 'message'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%m/%Y, %H:%M')
    return df

The `all_chats` dictionary holds the content of each file as a dataframe with three columns: `timestamp`, `sender`, and `message`.


In [24]:
from pathlib import Path

all_chats = {}
data_directory = Path("WhatsApp")
for file in data_directory.glob('*.txt'):
    file_name = file.stem
    all_chats[file_name] = read_whatsapp_chat(file)

## Text sequence


The text should be merged into a single sequence to prepare it for the next step, where the BPE algorithm will be applied and the text will be encoded.


In [25]:
text_sequence = ""
for file_name in all_chats.keys():
    text_sequence += " ".join(all_chats[file_name]['message'].values)

len(text_sequence)

0

In [21]:
with open("./output/combined_text.txt", "w") as f:
    f.write(text_sequence)

In [33]:
import re
import pandas as pd
from pathlib import Path

def read_whatsapp_chat(file_path: str) -> pd.DataFrame:
    with open(str(file_path), 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Remove system messages (media, encryption, deleted messages)
    filtered_lines = []
    for line in lines:
        if not any(exclude in line for exclude in [
            "Messages and calls are end-to-end encrypted",
            "<Media omitted>",
            "This message was deleted"
        ]):
            filtered_lines.append(line)

    # Join messages as a single text
    content = '\n'.join(filtered_lines)

    # Updated regex pattern to correctly extract messages
    pattern = r'\[(\d{2}/\d{2}/\d{2,4}), (\d{1,2}:\d{2}:\d{2}\s?[APM\u202F]*)\] (.*?): (.*?)(?=\n\[\d{2}/\d{2}/\d{2,4}, \d{1,2}:\d{2}:\d{2}\s?[APM\u202F]*\]|\Z)'

    messages = re.findall(pattern, content, re.DOTALL)

    # Convert to DataFrame
    df = pd.DataFrame(messages, columns=['date', 'time', 'sender', 'message'])
    df['timestamp'] = pd.to_datetime(df['date'] + " " + df['time'], format='%d/%m/%y %I:%M:%S %p')
    df.drop(columns=['date', 'time'], inplace=True)

    return df

# Read all WhatsApp chats
all_chats = {}
data_directory = Path("WhatsApp")

if not data_directory.exists():
    print("Error: WhatsApp directory does not exist!")
elif not list(data_directory.glob("*.txt")):
    print("Error: No .txt files found in WhatsApp directory!")
else:
    for file in data_directory.glob('*.txt'):
        file_name = file.stem
        all_chats[file_name] = read_whatsapp_chat(file)

# Merge all messages
text_sequence = " ".join(
    msg for chat in all_chats.values() for msg in chat['message'].values
)

# Save output
output_directory = Path("./output")
output_directory.mkdir(parents=True, exist_ok=True)

with open(output_directory / "combined_text.txt", "w", encoding="utf-8") as f:
    f.write(text_sequence)

print(f"Combined text saved to {output_directory}/combined_text.txt")


Combined text saved to output/combined_text.txt


In [39]:
import re
import json
import pandas as pd
from pathlib import Path

# List of names that should be considered as "assistant"
ASSISTANT_NAMES = ["Aditya Pandey", "Umyal Dixit"]

def read_whatsapp_chat(file_path: Path) -> pd.DataFrame:
    """Reads a WhatsApp chat file and converts it into a structured DataFrame."""
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Remove system messages (e.g., media omitted, encryption notices)
    filtered_lines = []
    for line in lines:
        if not any(exclude in line for exclude in [
            "Messages and calls are end-to-end encrypted",
            "<Media omitted>",
            "This message was deleted",
            "<This message was edited>"
        ]):
            filtered_lines.append(line.strip())

    # Combine messages into a single string
    content = '\n'.join(filtered_lines)

    # Updated regex pattern to handle different timestamp formats
    pattern = r'\[(\d{1,2}/\d{1,2}/\d{2,4}), (\d{1,2}:\d{2}:\d{2}[\s\u202F]?(?:AM|PM)?)\] (.*?): (.*?)(?=\n\[\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}:\d{2}[\s\u202F]?(?:AM|PM)?\]|\Z)'

    messages = re.findall(pattern, content, re.DOTALL)

    # Debugging: Print extracted messages
    if not messages:
        print(f"Warning: No messages extracted from {file_path}")

    # Convert to DataFrame
    df = pd.DataFrame(messages, columns=['date', 'time', 'sender', 'message'])
    df['timestamp'] = pd.to_datetime(df['date'] + " " + df['time'], errors='coerce')
    df.drop(columns=['date', 'time'], inplace=True)

    return df

# Read all WhatsApp chat files
data_directory = Path("WhatsApp")
all_chats = {}

if not data_directory.exists():
    print("Error: WhatsApp directory does not exist!")
elif not list(data_directory.glob("*.txt")):
    print("Error: No .txt files found in WhatsApp directory!")
else:
    for file in data_directory.glob('*.txt'):
        file_name = file.stem
        df = read_whatsapp_chat(file)
        if not df.empty:
            all_chats[file_name] = df
        else:
            print(f"Warning: No data extracted from {file_name}")

# Convert to Ollama training JSON format
ollama_data = []

for file_name, df in all_chats.items():
    messages = []

    for _, row in df.iterrows():
        role = "assistant" if row["sender"] in ASSISTANT_NAMES else "user"
        messages.append({"role": role, "content": row["message"]})

    if messages:
        ollama_data.append({
            "system": "You are a helpful AI trained on WhatsApp conversations.",
            "messages": messages
        })

# Ensure output directory exists
output_directory = Path("./output")
output_directory.mkdir(parents=True, exist_ok=True)

# Save JSON file
output_file = output_directory / "ollama_training.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(ollama_data, f, indent=4, ensure_ascii=False)

print(f"✅ Ollama training data saved to {output_file}")


/tmp/ipykernel_21338/169822204.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['date'] + " " + df['time'], errors='coerce')


✅ Ollama training data saved to output/ollama_training.json


In [42]:
import re
import pandas as pd
import os
import json

# Folder containing WhatsApp chat files
input_folder = "WhatsApp/"
output_folder = "output/"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Define assistant user
assistant_user = "Umyal Dixit"

# WhatsApp chat regex pattern
pattern = r"(\d{2}/\d{2}/\d{2}), (\d{1,2}:\d{2} ?[ap]m) - (.*?): (.*)"

# Store all messages
all_data = []

# Process all .txt files in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(input_folder, filename)
        print(f"🔄 Processing {filename}...")

        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, time, user, message = match.groups()
                role = "assistant" if user == assistant_user else "user"
                all_data.append({"timestamp": f"{date} {time}", "user": user, "message": message, "role": role})

# Convert to JSONL format
jsonl_output_file = os.path.join(output_folder, "ollama_training.jsonl")

with open(jsonl_output_file, "w", encoding="utf-8") as f:
    for entry in all_data:
        json.dump(entry, f, ensure_ascii=False)
        f.write("\n")

print(f"✅ Training data saved to {jsonl_output_file}")


🔄 Processing test.txt...
🔄 Processing 2.txt...
🔄 Processing ad.txt...
🔄 Processing 4.txt...
✅ Training data saved to output/ollama_training.jsonl


In [ ]:
import os
import re
import pandas as pd
from datasets import Dataset

# Define folder containing WhatsApp chat files
whatsapp_folder = "WhatsApp"

# WhatsApp message regex pattern (handles both AM/PM and 24-hour formats)
whatsapp_pattern = r"(\d{2}/\d{2}/\d{2,4}), (\d{1,2}:\d{2}\s?(?:AM|PM|am|pm)?) - ([^:]+): (.+)"

# Store extracted data
data = []

# Loop through all .txt files in the folder
for filename in os.listdir(whatsapp_folder):
    file_path = os.path.join(whatsapp_folder, filename)

    if filename.endswith(".txt"):  # Process only text files
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

        # Store the last user message to match with the assistant's reply
        last_user_message = None
        last_user_name = None

        for line in lines:
            match = re.match(whatsapp_pattern, line)
            if match:
                date, time, sender, message = match.groups()

                # Skip system messages
                if "Messages and calls are end-to-end encrypted" in message:
                    continue

                # Assign roles
                role = "assistant" if sender.strip() == "Umyal Dixit" else "user"

                if role == "user":
                    last_user_message = message.strip()
                    last_user_name = sender.strip()
                elif role == "assistant" and last_user_message:
                    # Create Question-Answer pair
                    data.append({
                        "question": f"{last_user_name}: {last_user_message}",
                        "answer": f"Umyal Dixit: {message.strip()}"
                    })
                    last_user_message = None  # Reset for next pair

# Convert extracted data to a DataFrame
df = pd.DataFrame(data)

# Convert DataFrame to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

# Format for training
dataset = dataset.map(lambda x: {"text": f"### Question: {x['question']}\n### Answer: {x['answer']}"})

# Save formatted dataset
df.to_csv("whatsapp_dataset.csv", index=False)
dataset.to_json("whatsapp_dataset.json", orient="records")

print("✅ WhatsApp chat data formatted and saved!")


Creating json from Arrow format: 100%|██████████| 9/9 [00:00<00:00, 489.73ba/s]

✅ WhatsApp chat data formatted and saved!


In [3]:
import os
import torch
import gc
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Step 1: Load and Format WhatsApp Dataset
df = pd.read_csv('whatsapp_dataset.csv')  # Ensure the dataset is in CSV format
dataset = Dataset.from_pandas(df)
dataset = dataset.map(lambda x: {"text": f"### Question: {x['question']}\n### Answer: {x['answer']}"})

# Step 2: Initialize Dolphin 3 Model & Tokenizer
model_name = "dolphin3"  # Replace with Dolphin-3 when available
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# LoRA Configuration for Efficient Fine-Tuning
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    task_type="CAUSAL_LM"
)

# Load Base Model with Quantization (for lower VRAM usage)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(load_in_4bit=True)
)

# Step 3: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./dolphin3_finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # Change to "wandb" if using Weights & Biases
)

# Step 4: Fine-Tune the Model
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_args
)

trainer.train()

# Step 5: Save Fine-Tuned Model Locally
trainer.model.save_pretrained('finetuned_dolphin3')

# Step 6: Evaluate & Inference
pipe = pipeline('text-generation', model=trainer.model, tokenizer=tokenizer)
output = pipe("### Question: What is your name?")[0]['generated_text']
print(output)

# Step 7: Free Up Memory
del base_model, trainer, pipe
gc.collect()

# Step 8: Merge LoRA with Base Model & Upload to Hugging Face Hub
base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base_model, 'finetuned_dolphin3')
model = model.merge_and_unload()

# Step 9: Push Fine-Tuned Model to Hugging Face Hub
# model.push_to_hub("your_huggingface_username/finetuned_dolphin3")
# tokenizer.push_to_hub("your_huggingface_username/finetuned_dolphin3")

print("✅ Fine-tuned Dolphin 3 model saved & uploaded!")



===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('0'), PosixPath('1')}
The following directories listed in your path were found to be non-existent: {PosixPath('@/tmp/.ICE-unix/2009,unix/pop-os'), PosixPath('local/pop-os')}
The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {PosixPath('{}}'), PosixPath('"en","defaultMessagesFile"'), PosixPath('"en-us","osLocale"'), PosixPath('{"userLocale"'), PosixPath('"en-us","availableLanguages"'), PosixPath('"/usr/share/code/resources/app/out/nls.messages.json","locale"'), PosixPath('"en-us","resolvedLanguage"')}
The following directories listed in your path were found to be non-existent: {PosixPath('/etc/xdg/xdg-pop')}
The following directories listed in your path were fo

RuntimeError: Failed to import transformers.training_args because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues